In [66]:
from scipy import signal
from scipy import linalg
import scipy.io.wavfile
import numpy as np
import os


path_db = '05_AudioClassifier_Pdf/05_AudioClassifier_Pdf/database/'
path_music = path_db+'music/'
path_speech = path_db+'speech/'
path_unknowns = path_db.replace('database/','')+'unknownSounds/'

spg_music = 0
spg_speech = 0


def load_audio(path):
    audio_list = []#in py questa è una semplice lista vuota
    for filename in os.listdir(path):
        audio_list.append(scipy.io.wavfile.read(path+filename))
    return audio_list
    
def spg(path):
    start = True
    _spg = 0
    for filename in os.listdir(path):
        rate,data = scipy.io.wavfile.read(path+filename)
        f, t, Sxx = signal.spectrogram(data, rate, 'hamming',512,160,512)
        if start:
            _spg = Sxx
            start=False
        else:
            _spg = scipy.hstack((_spg,Sxx))
    return _spg

def spg_b(s,V):
    #s è il vettore(sarebbe matrice ma diagonale) dei valori singolari
    #V è la matrice dei vettori destri (la funzione vuole V.T=>autovettori per colonna)
    ssum = np.sum(s)
    approx_sum = 0
    i = 0;
    for e_value in s:
        approx_sum = approx_sum + e_value
        if approx_sum >= ssum*0.9:
            break
        i = i+1
    base = V[:,range(0,i)]
    return base
    

In [67]:

spg_music = spg(path_music).T
#traspose spg means that the columns of V are eigenvector for M*M.H(oppure M.H*M)
print(spg_music.shape)
spg_music = np.delete(spg_music,256,0)
U, s, V= np.linalg.svd(spg_music)
b_music = spg_b(s,V)
#print(b_music.shape)
#print(np.linalg.matrix_rank(b_music))


(9080, 257)


In [68]:
spg_speech = spg(path_speech).T
print(spg_speech.shape)
spg_speech = np.delete(spg_speech,256,0)
U, s, V= np.linalg.svd(spg_speech)
b_speech = spg_b(s,V)
#print(b_speech.shape)
#print(np.linalg.matrix_rank(b_speech))

(2980, 257)


In [69]:
unknowns = load_audio(path_unknowns)
unknowns

[(16000, array([ 1520,  1164,  1560, ...,  -232,  -940, -1979], dtype=int16)),
 (16000, array([17, 24, 15, ...,  5,  7, -1], dtype=int16)),
 (16000, array([1, 2, 2, ..., 1, 0, 5], dtype=int16)),
 (16000, array([-1617, -1563, -1473, ..., -3358, -3364, -2902], dtype=int16)),
 (16000, array([2, 3, 2, ..., 3, 0, 0], dtype=int16)),
 (16000, array([2, 3, 3, ..., 6, 6, 6], dtype=int16)),
 (16000, array([ 1424,  6801,  8511, ..., -2669,  1359, -1264], dtype=int16)),
 (16000, array([-2609, -2725, -2910, ...,  3971,  4190,  4345], dtype=int16))]

In [70]:
for rate,data in unknowns:
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    spg_unk = Sxx
    print(spg_speech.shape,"*",Sxx.shape,'=',np.dot(spg_speech,Sxx).shape) 
    print(spg_music.shape,"*",Sxx.shape,'=',np.dot(spg_music,Sxx).shape) 
    unk_on_music = np.dot(spg_speech,Sxx)
    unk_on_speech = np.dot(spg_music,Sxx)
    
    #posso fare i prodotto per fortuna
    # ma il prodotto * si arrabia 
    # e ho dovuto usaer np.dot() ... For 2-D arrays it is equivalent to matrix multiplication
    #dato_music = np.linalg.norm(unk_on_music)
    #dato_speech = np.linalg.norm(unk_on_speech)
    #print('NormaSpeech: ',dato_speech,'\nNormMusic: ',dato_music)
    

(2979, 257) * (257, 454) = (2979, 454)
(9079, 257) * (257, 454) = (9079, 454)
NormaSpeech:  2.11142e+13 
NormMusic:  5.22425e+11
(2979, 257) * (257, 210) = (2979, 210)
(9079, 257) * (257, 210) = (9079, 210)
NormaSpeech:  3.13934e+11 
NormMusic:  1.86522e+10
(2979, 257) * (257, 220) = (2979, 220)
(9079, 257) * (257, 220) = (9079, 220)
NormaSpeech:  1.28197e+11 
NormMusic:  7.53015e+09
(2979, 257) * (257, 454) = (2979, 454)
(9079, 257) * (257, 454) = (9079, 454)
NormaSpeech:  1.52091e+14 
NormMusic:  2.43101e+11
(2979, 257) * (257, 214) = (2979, 214)
(9079, 257) * (257, 214) = (9079, 214)
NormaSpeech:  7.14153e+11 
NormMusic:  3.91266e+10
(2979, 257) * (257, 71) = (2979, 71)
(9079, 257) * (257, 71) = (9079, 71)
NormaSpeech:  1.1907e+11 
NormMusic:  6.68942e+09
(2979, 257) * (257, 454) = (2979, 454)
(9079, 257) * (257, 454) = (9079, 454)
NormaSpeech:  1.31176e+14 
NormMusic:  2.95085e+11
(2979, 257) * (257, 454) = (2979, 454)
(9079, 257) * (257, 454) = (9079, 454)
NormaSpeech:  1.13298e+1